In [380]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 
from sklearn.metrics import accuracy_score, roc_auc_score
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, RocCurveDisplay, make_scorer
from sklearn.model_selection import learning_curve
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import TomekLinks
import datetime
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

import warnings
warnings.filterwarnings('ignore')

In [381]:
#data_transformed = pd.read_csv('new_data/clean-data_without_outliers.csv')
#competition_transformed = pd.read_csv('new_data/clean-comp.csv')
#data = pd.read_csv('new_data/clean-data_without_outliers.csv')
competition = pd.read_csv('new_data/clean-comp.csv')

data = pd.read_csv('new_data/complete-data.csv')
#data_transformed

In [382]:
data.sort_values('year')

,playerID,year,stint,tmID,GP,oRebounds,dRebounds,dq,PostMinutes,PostPoints,PostoRebounds,PostdRebounds,PostRebounds,PostAssists,PostSteals,PostBlocks,PostTurnovers,PostPF,PostDQ,ft%,fg%,three%,gs%,Postft%,Postfg%,Postthree%,Postgs%,efg%,ts%,ppg,rpg,apg,spg,bpg,eff,pp36,defensive_prowess,defensive_discipline,mpg,pos,college,playoff,confID,playoff_progression,height,weight,award_count,career_year,playoff_progression_rolling,playoff_rolling,pp36_rolling,eff_rolling,award_count_rolling,defensive_prowess_rolling,defensive_discipline_rolling
300,191,2,2,569,13.0,0.38,0.85,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.57,0.41,0.00,0.00,0.0,0.0,0.0,0.0,0.82,0.90,2.77,1.23,0.31,0.31,0.00,-28.38,10.44,11.5,2.76,9.46,2,615,0.0,702,0,73.0,180,0.0,3,1.5,1.0,9.90,-86.415,0.0,24.90,5.02
720,393,2,0,567,28.0,1.18,1.96,0.00,7.67,0.33,1.0,1.0,2.0,0.0,0.33,0.0,2.0,0.33,0.0,0.75,0.36,0.25,0.11,0.5,0.0,0.0,0.0,0.74,0.80,2.68,3.14,0.57,0.39,0.50,-82.72,6.48,28.6,5.64,14.86,2,612,1.0,702,1,76.0,174,0.0,3,0.0,0.0,5.04,-44.760,0.0,28.00,6.04
223,136,2,1,573,10.0,0.50,1.40,0.10,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.79,0.37,0.36,0.90,0.0,0.0,0.0,0.0,0.80,1.02,9.40,1.90,1.80,0.50,0.10,-63.30,16.20,20.0,10.40,20.80,1,588,0.0,701,0,70.0,160,0.0,3,2.0,1.0,11.52,-181.040,0.0,27.75,8.52
218,133,2,0,563,3.0,0.00,0.67,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,1.00,0.33,0.00,0.00,0.0,0.0,0.0,0.0,0.66,1.04,1.33,0.67,1.33,0.33,0.00,-1.34,11.16,10.0,3.34,4.33,1,578,0.0,701,0,69.0,150,0.0,3,1.0,0.5,13.68,-68.565,0.0,14.85,5.68
615,343,3,0,561,24.0,0.12,1.08,0.04,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.44,0.27,0.21,0.33,0.0,0.0,0.0,0.0,0.62,0.66,1.42,1.21,3.04,0.42,0.17,-65.74,2.88,16.7,6.84,17.04,1,658,0.0,701,0,67.0,147,0.0,3,1.5,1.0,5.40,-67.345,0.0,20.00,6.87
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,231,11,0,745,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,2,615,0.0,701,0,75.0,185,0.0,4,1.0,0.5,14.04,-133.615,0.0,42.50,6.64
835,442,11,0,562,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,1,588,0.0,702,0,75.0,183,0.0,4,1.0,0.5,12.24,-82.975,0.0,13.65,3.15
169,111,11,0,559,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,2,598,0.0,702,0,72.0,177,0.0,6,0.5,0.5,14.22,-248.860,0.0,44.40,10.24
356,218,11,0,745,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,2,649,0.0,701,0,72.0,168,0.0,3,0.5,0.5,12.06,-120.655,0.0,28.60,5.18


In [383]:
competition.columns

Index(['playerID', 'year', 'stint', 'tmID', 'height', 'weight', 'pos',
       'college', 'confID'],
      dtype='object')

In [384]:
input_cols = ['playerID', 'year','stint', 'tmID', 'pos', 'college', 'height', 'weight', 'career_year', 'confID']

#add all columns ending in rolling from data to input cols
input_cols+=[c for c in data.columns if c.endswith("_rolling")]

# The output columns are the genres
output_cols = 'playoff'

known_columns = ['playerID', 'year', 'stint', 'tmID', 'height', 'weight', 'pos','college', 'confID']

# Averages to calculate for precision, recall, and f1-score
averages = [None, "macro", "weighted", "micro", "samples"]

print(input_cols)

['playerID', 'year', 'stint', 'tmID', 'pos', 'college', 'height', 'weight', 'career_year', 'confID', 'playoff_progression_rolling', 'playoff_rolling', 'pp36_rolling', 'eff_rolling', 'award_count_rolling', 'defensive_prowess_rolling', 'defensive_discipline_rolling']


In [385]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)
data

,playerID,year,stint,tmID,GP,oRebounds,dRebounds,dq,PostMinutes,PostPoints,PostoRebounds,PostdRebounds,PostRebounds,PostAssists,PostSteals,PostBlocks,PostTurnovers,PostPF,PostDQ,ft%,fg%,three%,gs%,Postft%,Postfg%,Postthree%,Postgs%,efg%,ts%,ppg,rpg,apg,spg,bpg,eff,pp36,defensive_prowess,defensive_discipline,mpg,pos,college,playoff,confID,playoff_progression,height,weight,award_count,career_year,playoff_progression_rolling,playoff_rolling,pp36_rolling,eff_rolling,award_count_rolling,defensive_prowess_rolling,defensive_discipline_rolling
0,0,4,0,555,30.0,1.47,3.23,0.00,23.0,7.67,0.33,1.33,1.67,1.33,1.33,0.33,2.67,2.67,0.0,0.70,0.39,0.30,0.83,1.00,0.27,0.43,1.0,0.88,0.96,10.60,4.70,2.73,1.47,0.37,-272.13,14.40,50.7,11.26,26.40,2,575,1.0,701,1,74.0,169,0.0,3,0.0,0.0,14.40,-301.960,0.0,63.35,12.07
1,0,5,0,555,22.0,0.77,2.59,0.00,33.5,10.00,1.50,3.00,4.50,1.50,0.50,1.00,1.50,3.50,0.0,0.61,0.35,0.38,0.50,0.50,0.35,0.25,1.0,0.84,0.92,6.64,3.36,2.05,1.36,0.09,-137.50,11.52,40.5,7.72,21.00,2,575,1.0,701,1,74.0,169,0.0,4,0.5,0.5,14.22,-299.970,0.0,53.70,11.74
2,0,6,0,555,31.0,0.94,2.52,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.73,0.39,0.40,1.00,0.00,0.00,0.00,0.0,0.90,0.98,9.81,3.45,1.94,1.55,0.19,-250.06,14.04,42.6,10.70,25.06,2,575,0.0,701,0,74.0,169,0.0,5,1.0,1.0,12.96,-204.815,0.0,45.60,9.49
3,0,7,0,555,34.0,1.29,1.82,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.66,0.41,0.37,0.06,0.00,0.00,0.00,0.0,0.92,0.98,7.74,3.12,1.59,1.00,0.03,-210.52,12.96,28.5,7.48,21.29,2,575,0.0,701,0,74.0,169,0.0,6,0.5,0.5,12.78,-193.780,0.0,41.55,9.21
4,0,8,0,555,34.0,1.56,2.85,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.84,0.44,0.45,0.85,0.00,0.00,0.00,0.0,1.04,1.08,10.15,4.41,2.50,1.29,0.12,-222.53,14.76,42.6,9.06,24.79,2,575,0.0,701,0,74.0,169,0.0,7,0.0,0.0,13.50,-230.290,0.0,35.55,9.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1024,548,10,0,564,33.0,2.09,4.42,0.03,32.0,19.33,2.00,3.33,5.33,2.00,2.00,0.67,1.67,3.00,0.0,0.77,0.45,0.31,1.00,0.68,0.46,0.50,1.0,0.94,1.06,18.18,6.52,1.64,1.33,0.48,-340.85,19.44,62.4,7.82,33.73,2,634,1.0,701,1,73.0,165,0.0,4,2.5,1.0,18.90,-305.255,0.0,58.50,9.21
1025,548,11,0,564,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,2,634,0.0,701,0,73.0,165,0.0,5,2.0,1.0,19.62,-320.180,0.0,62.10,8.24
1026,549,10,2,574,10.0,1.00,2.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.89,0.35,0.25,0.60,0.00,0.00,0.00,0.0,0.74,0.92,5.00,3.00,0.70,0.70,0.40,-33.20,14.04,31.0,5.80,12.90,2,700,0.0,702,0,74.0,166,0.0,3,0.5,0.5,13.14,-131.490,0.0,27.25,6.37
1027,549,11,0,574,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,2,700,0.0,702,0,74.0,166,0.0,4,0.5,0.5,14.40,-30.555,0.0,22.30,4.54


In [386]:
data.isnull().any()

playerID                        False
year                            False
stint                           False
tmID                            False
GP                              False
                                ...  
pp36_rolling                    False
eff_rolling                     False
award_count_rolling             False
defensive_prowess_rolling       False
defensive_discipline_rolling    False
Length: 55, dtype: bool

In [387]:
# Adapted from https://scikit-learn.org/stable/auto_examples/model_selection/plot_learning_curve.html
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve
from imblearn.pipeline import Pipeline

def plot_learning_curve(
    title,
    train_sizes, 
    train_scores, 
    test_scores, 
    fit_times,
    score_times,
    axes=None,
    ylim=None,
):
    """
    Generate 3 plots: the test and training learning curve, the training
    samples vs fit times curve, the fit times vs score curve.

    Parameters
    ----------
    estimator : estimator instance
        An estimator instance implementing `fit` and `predict` methods which
        will be cloned for each validation.

    title : str
        Title for the chart.

    X : array-like of shape (n_samples, n_features)
        Training vector, where ``n_samples`` is the number of samples and
        ``n_features`` is the number of features.

    y : array-like of shape (n_samples) or (n_samples, n_features)
        Target relative to ``X`` for classification or regression;
        None for unsupervised learning.

    axes : array-like of shape (3,), default=None
        Axes to use for plotting the curves.

    ylim : tuple of shape (2,), default=None
        Defines minimum and maximum y-values plotted, e.g. (ymin, ymax).

    cv : int, cross-validation generator or an iterable, default=None
        Determines the cross-validation splitting strategy.
        Possible inputs for cv are:

          - None, to use the default 5-fold cross-validation,
          - integer, to specify the number of folds.
          - :term:`CV splitter`,
          - An iterable yielding (train, test) splits as arrays of indices.

        For integer/None inputs, if ``y`` is binary or multiclass,
        :class:`StratifiedKFold` used. If the estimator is not a classifier
        or if ``y`` is neither binary nor multiclass, :class:`KFold` is used.

        Refer :ref:`User Guide <cross_validation>` for the various
        cross-validators that can be used here.

    n_jobs : int or None, default=None
        Number of jobs to run in parallel.
        ``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.
        ``-1`` means using all processors. See :term:`Glossary <n_jobs>`
        for more details.

    train_sizes : array-like of shape (n_ticks,)
        Relative or absolute numbers of training examples that will be used to
        generate the learning curve. If the ``dtype`` is float, it is regarded
        as a fraction of the maximum size of the training set (that is
        determined by the selected validation method), i.e. it has to be within
        (0, 1]. Otherwise it is interpreted as absolute sizes of the training
        sets. Note that for classification the number of samples usually have
        to be big enough to contain at least one sample from each class.
        (default: np.linspace(0.1, 1.0, 5))
    """
    if axes is None:
        fig, axes = plt.subplots(3, 2, figsize=(10, 15))

    axes = axes.reshape(-1)
    fig.subplots_adjust(hspace=0.5, wspace=0.5)
    fig = fig.delaxes(axes[-1])
    
    axes[0].set_title(title)
    if ylim is not None:
        axes[0].set_ylim(*ylim)
    axes[0].set_xlabel("Training examples")
    axes[0].set_ylabel("Score")

    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    fit_times_mean = np.mean(fit_times, axis=1)
    fit_times_std = np.std(fit_times, axis=1)
    score_times_mean = np.mean(score_times, axis=1)
    score_times_std = np.std(score_times, axis=1)

    # Plot learning curve
    axes[0].grid()
    axes[0].fill_between(
        train_sizes,
        train_scores_mean - train_scores_std,
        train_scores_mean + train_scores_std,
        alpha=0.1,
        color="r",
    )
    axes[0].fill_between(
        train_sizes,
        test_scores_mean - test_scores_std,
        test_scores_mean + test_scores_std,
        alpha=0.1,
        color="g",
    )
    axes[0].plot(
        train_sizes, train_scores_mean, "o-", color="r", label="Training score"
    )
    axes[0].plot(
        train_sizes, test_scores_mean, "o-", color="g", label="Cross-validation score"
    )
    axes[0].legend(loc="best")

    # Plot n_samples vs fit_times
    axes[1].grid()
    axes[1].plot(train_sizes, fit_times_mean, "o-")
    axes[1].fill_between(
        train_sizes,
        fit_times_mean - fit_times_std,
        fit_times_mean + fit_times_std,
        alpha=0.1,
    )
    axes[1].set_xlabel("Training examples")
    axes[1].set_ylabel("fit_times")
    axes[1].set_title("Scalability of the model")

    # Plot fit_time vs score
    fit_time_argsort = fit_times_mean.argsort()
    fit_time_sorted = fit_times_mean[fit_time_argsort]
    test_scores_mean_sorted = test_scores_mean[fit_time_argsort]
    test_scores_std_sorted = test_scores_std[fit_time_argsort]
    axes[2].grid()
    axes[2].plot(fit_time_sorted, test_scores_mean_sorted, "o-")
    axes[2].fill_between(
        fit_time_sorted,
        test_scores_mean_sorted - test_scores_std_sorted,
        test_scores_mean_sorted + test_scores_std_sorted,
        alpha=0.1,
    )
    axes[2].set_xlabel("fit_times")
    axes[2].set_ylabel("Score")
    axes[2].set_title("Performance of the model")

    # Plot n_samples vs score_times
    axes[3].grid()
    axes[3].plot(train_sizes, score_times_mean, "o-")
    axes[3].fill_between(
        train_sizes,
        score_times_mean - score_times_std,
        score_times_mean + score_times_std,
        alpha=0.1,
    )
    axes[3].set_xlabel("Training examples")
    axes[3].set_ylabel("score_times")
    axes[3].set_title("Scalability of the model")

    # Plot score_time vs score
    score_time_argsort = score_times_mean.argsort()
    score_time_sorted = score_times_mean[score_time_argsort]
    test_scores_mean_sorted = test_scores_mean[score_time_argsort]
    test_scores_std_sorted = test_scores_std[score_time_argsort]
    axes[4].grid()
    axes[4].plot(score_time_sorted, test_scores_mean_sorted, "o-")
    axes[4].fill_between(
        score_time_sorted,
        test_scores_mean_sorted - test_scores_std_sorted,
        test_scores_mean_sorted + test_scores_std_sorted,
        alpha=0.1,
    )
    axes[4].set_xlabel("score_times")
    axes[4].set_ylabel("Score")
    axes[4].set_title("Performance of the model")

    return plt

In [388]:
model = DecisionTreeClassifier()
#loop in rolling window method through all data
unique_years = data['year'].unique()
#sort the years
unique_years.sort()
rolling_window_results = []
competition_predictions = []
trained_models = list()



print(unique_years)


[ 2  3  4  5  6  7  8  9 10 11]


In [389]:
def get_columns_to_remove():
    return ['playoff']

In [390]:

toZeroArr = [c for c in data.columns if c not in input_cols]
#add playoff to the list of columns to remove
toZeroArr.remove('playoff')
print(toZeroArr)

['GP', 'oRebounds', 'dRebounds', 'dq', 'PostMinutes', 'PostPoints', 'PostoRebounds', 'PostdRebounds', 'PostRebounds', 'PostAssists', 'PostSteals', 'PostBlocks', 'PostTurnovers', 'PostPF', 'PostDQ', 'ft%', 'fg%', 'three%', 'gs%', 'Postft%', 'Postfg%', 'Postthree%', 'Postgs%', 'efg%', 'ts%', 'ppg', 'rpg', 'apg', 'spg', 'bpg', 'eff', 'pp36', 'defensive_prowess', 'defensive_discipline', 'mpg', 'playoff_progression', 'award_count']


In [391]:
def prepare_training_data(data, unique_years, i):
    train_years_before = unique_years[:i]
    train_years = unique_years[:i+1]
    test_year = unique_years[i]

    if i + 1 > len(unique_years): 
        return

    predict_year = unique_years[i+1]  # Predicting for year i+2 

    if predict_year < test_year:
        return

    # print("Train years:", train_years)
    # print("Predict Year: ", predict_year)
    
    train_before = data[data['year'].isin(train_years_before)]
    train = data[data['year'].isin(train_years)]
    test = data[data['year'] == test_year]
    actual = data[data['year'] == predict_year]

    return train_before, train, test, actual, predict_year, train_years, test_year


In [392]:

def process_data(train_before, train, test, actual, data, unique_years):
    #one hot encoding
    # train_before = pd.get_dummies(train_before, columns=['tmID', 'confID', 'pos', 'college', 'playerID'])
    # train = pd.get_dummies(train, columns=['tmID', 'confID', 'pos', 'college', 'playerID']) 
    # Removing and adding rows based on certain conditions
    # train_before_indices_to_remove = []
    # train_indices_to_remove = []
    # #check if a player have a entry in the train df with the next year
    # for index, row in train_before.iterrows():
    #     player_id = row['playerID']
    #     year = row['year'] + 1 
    #     if row['stint'] > 1:
    #         train_before_indices_to_remove.append(index)
    #     elif not (((train['playerID'] == player_id) & (train['year'] == year)).any()):
    #         data_to_add = {
    #             'playerID': player_id, 
    #             'year': year,        
    #         }
    #         new_row = pd.DataFrame(data_to_add, index=[0])
    #         train = pd.concat([train, new_row], ignore_index=True)
       

    # train_before = train_before.drop(train_before_indices_to_remove)
    # for index, row in train.iterrows():
    #     player_id = row['playerID']
    #     year = row['year'] - 1 
    #     if row['stint'] > 1:
    #         train_indices_to_remove.append(index)
    #     elif not (((train_before['playerID'] == player_id) & (train_before['year'] == year)).any()):
    #         data_to_add = {
    #             'playerID': player_id, 
    #             'year': year,        
    #         }
    #         new_row = pd.DataFrame(data_to_add, index=[0])
    #         train_before = pd.concat([train_before , new_row], ignore_index=True)

    # train = train_before.drop(train_indices_to_remove)
    # train.fillna(0, inplace=True)
    # train.sort_values(by='playerID', inplace=True)
    # train_before.fillna(0, inplace=True)
    # train_before.sort_values(by='playerID', inplace=True)
    remove_columns = get_columns_to_remove()
    actual[toZeroArr] = 0

    #remove stint > 0
    train = train[train['stint'] == 0]
    actual = actual[actual['stint'] == 0]


    X_train = train.drop(remove_columns, axis=1)
    y_train = train[output_cols]
    X_actual = actual.drop(remove_columns, axis=1)
    y_actual = actual[output_cols]

    return X_train, y_train, X_actual, y_actual


In [393]:

def train_and_evaluate_model(X_train, y_train, model):
    classifier = model.fit(X_train, y_train)
    return classifier


In [394]:

def predict_and_evaluate(classifier, X_actual, y_actual):
    accuracy = 0.0
    auc_score = 0.0
    # Evaluate the model
    predictions = classifier.predict(X_actual)
    predictions_prob = classifier.predict_proba(X_actual)[:, 1]

    if X_actual['year'].unique()[0]==11:
        competition_predictions.append(predictions)
    else:
        accuracy = accuracy_score(y_actual, predictions)
        auc_score = roc_auc_score(y_actual, predictions_prob)

    return accuracy, auc_score, predictions, predictions_prob


In [395]:

def rolling_window_method(i, model, data, unique_years):
    train_before, train, test, actual, predict_year, train_years, test_year = prepare_training_data(data, unique_years, i)
    
    if train_before is None:
        return None

    X_train, y_train, X_actual, y_actual= process_data(train_before, train, test, actual, data, unique_years)

    #print years in X_train and X_actual
    print("Unique years in X_train:", X_train['year'].unique())
    print("Unique years in X_actual:", X_actual['year'].unique())
    
    #create a csv file for each year, storing the data for that year and that model, in the folder output/rolling_window
    #X_train.to_csv(f'output/rolling_window/{model.__class__.__name__}_{test_year}_train.csv', index=False)
    #X_test.to_csv(f'output/rolling_window/{model.__class__.__name__}_{test_year}_test.csv', index=False)
    #X_actual.to_csv(f'output/rolling_window/{model.__class__.__name__}_{test_year}_actual.csv', index=False)
    # y_actual.to_csv(f'output/rolling_window/{model.__class__.__name__}_{test_year}_y_actual.csv', index=False)


    classifier = train_and_evaluate_model(X_train, y_train, model)

    # Calculate accuracy

    accuracy, auc_score, predictions, predictions_prob = predict_and_evaluate(classifier, X_actual, y_actual)

    #print(f"Best params for {model.__class__.__name_}: {model.best_params_}")
    # print(f"Accuracy: {accuracy:.4f}")
    # print(f"AUC: {auc_score:.4f}")


    return {
        'train_years': train_years,
        'test_year': test_year,
        'Year': predict_year,
        'accuracy': accuracy,
        'auc': auc_score,
        'model': classifier,
        'predictions': predictions,
        'predictions_prob': predictions_prob,
        'actual_results': y_actual,
    }

In [396]:

def model_training_loop(model, data, unique_years):
    rolling_window_results = []
    for i in range(len(unique_years) - 1):
        result= rolling_window_method(i, model, data, unique_years)
        if result:
            rolling_window_results.append(result)
    return rolling_window_results

In [397]:
# model = DecisionTreeClassifier()
# rolling_window_results += model_training_loop(model, data, unique_years)
# trained_models.append(rolling_window_results[-1]['model'])  # Get the last trained model

In [398]:
model = RandomForestClassifier()
rolling_window_results += model_training_loop(model, data, unique_years)
trained_models.append(rolling_window_results[-1]['model'])  # Get the last trained model

Unique years in X_train: [2]
Unique years in X_actual: [3]
Unique years in X_train: [3 2]
Unique years in X_actual: [4]
Unique years in X_train: [4 3 2]
Unique years in X_actual: [5]
Unique years in X_train: [4 5 3 2]
Unique years in X_actual: [6]
Unique years in X_train: [4 5 6 3 2]
Unique years in X_actual: [7]
Unique years in X_train: [4 5 6 7 3 2]
Unique years in X_actual: [8]
Unique years in X_train: [4 5 6 7 8 3 2]
Unique years in X_actual: [9]
Unique years in X_train: [4 5 6 7 8 9 3 2]
Unique years in X_actual: [10]
Unique years in X_train: [ 4  5  6  7  8  9  3 10  2]
Unique years in X_actual: [11]


In [399]:
model = GradientBoostingClassifier()
rolling_window_results += model_training_loop(model, data, unique_years)
trained_models.append(rolling_window_results[-1]['model'])  # Get the last trained model

Unique years in X_train: [2]
Unique years in X_actual: [3]
Unique years in X_train: [3 2]
Unique years in X_actual: [4]
Unique years in X_train: [4 3 2]
Unique years in X_actual: [5]
Unique years in X_train: [4 5 3 2]
Unique years in X_actual: [6]
Unique years in X_train: [4 5 6 3 2]
Unique years in X_actual: [7]
Unique years in X_train: [4 5 6 7 3 2]
Unique years in X_actual: [8]
Unique years in X_train: [4 5 6 7 8 3 2]
Unique years in X_actual: [9]
Unique years in X_train: [4 5 6 7 8 9 3 2]
Unique years in X_actual: [10]
Unique years in X_train: [ 4  5  6  7  8  9  3 10  2]
Unique years in X_actual: [11]


In [400]:
#gaussian naive bayes
model = GaussianNB()
rolling_window_results += model_training_loop(model, data, unique_years)
trained_models.append(rolling_window_results[-1]['model'])  # Get the last trained model

Unique years in X_train: [2]
Unique years in X_actual: [3]
Unique years in X_train: [3 2]
Unique years in X_actual: [4]
Unique years in X_train: [4 3 2]
Unique years in X_actual: [5]
Unique years in X_train: [4 5 3 2]
Unique years in X_actual: [6]
Unique years in X_train: [4 5 6 3 2]
Unique years in X_actual: [7]
Unique years in X_train: [4 5 6 7 3 2]
Unique years in X_actual: [8]
Unique years in X_train: [4 5 6 7 8 3 2]
Unique years in X_actual: [9]
Unique years in X_train: [4 5 6 7 8 9 3 2]
Unique years in X_actual: [10]
Unique years in X_train: [ 4  5  6  7  8  9  3 10  2]
Unique years in X_actual: [11]


In [401]:
#logistic regression
model = LogisticRegression()
rolling_window_results += model_training_loop(model, data, unique_years)
trained_models.append(rolling_window_results[-1]['model'])  # Get the last trained model

Unique years in X_train: [2]
Unique years in X_actual: [3]


Unique years in X_train: [3 2]
Unique years in X_actual: [4]
Unique years in X_train: [4 3 2]
Unique years in X_actual: [5]
Unique years in X_train: [4 5 3 2]
Unique years in X_actual: [6]
Unique years in X_train: [4 5 6 3 2]
Unique years in X_actual: [7]
Unique years in X_train: [4 5 6 7 3 2]
Unique years in X_actual: [8]
Unique years in X_train: [4 5 6 7 8 3 2]
Unique years in X_actual: [9]
Unique years in X_train: [4 5 6 7 8 9 3 2]
Unique years in X_actual: [10]
Unique years in X_train: [ 4  5  6  7  8  9  3 10  2]
Unique years in X_actual: [11]


In [402]:
#ada boost
model = AdaBoostClassifier()
rolling_window_results += model_training_loop(model, data, unique_years)
trained_models.append(rolling_window_results[-1]['model'])  # Get the last trained model

Unique years in X_train: [2]
Unique years in X_actual: [3]


Unique years in X_train: [3 2]
Unique years in X_actual: [4]
Unique years in X_train: [4 3 2]
Unique years in X_actual: [5]
Unique years in X_train: [4 5 3 2]
Unique years in X_actual: [6]
Unique years in X_train: [4 5 6 3 2]
Unique years in X_actual: [7]
Unique years in X_train: [4 5 6 7 3 2]
Unique years in X_actual: [8]
Unique years in X_train: [4 5 6 7 8 3 2]
Unique years in X_actual: [9]
Unique years in X_train: [4 5 6 7 8 9 3 2]
Unique years in X_actual: [10]
Unique years in X_train: [ 4  5  6  7  8  9  3 10  2]
Unique years in X_actual: [11]


In [404]:
level0 = list()
#append models in trained_models to level0
for i in range(len(trained_models)):
    level0.append((str(i), trained_models[i]))

level1 = RandomForestClassifier(n_estimators=100, min_samples_split=2, min_samples_leaf=2, max_features='sqrt', max_depth=9, bootstrap=True, criterion='entropy', random_state=20)
clf = StackingClassifier(estimators=level0, final_estimator=level1, cv=3)

rolling_window_results += model_training_loop(clf, data, unique_years)
trained_models.append(rolling_window_results[-1]['model'])  # Get the last trained model

Unique years in X_train: [2]
Unique years in X_actual: [3]


ValueError: Cannot have number of splits n_splits=3 greater than the number of samples: n_samples=2.

In [405]:
# {
#         'train_years': train_years,
#         'test_year': test_year,
#         'Year': predict_year,
#         'accuracy': accuracy,
#         'auc': auc_score,
#         'model': classifier,
#         'predictions': predictions,
#         'predictions_prob': predictions_prob,
#         'actual_results': y_actual,
# }

#make a dictionary of the model: [accuracy]
model_accuracy = dict()
for result in rolling_window_results:
    model_name = result['model'].__class__.__name__
    if model_name not in model_accuracy:
        model_accuracy[model_name] = []
    model_accuracy[model_name].append(result['accuracy'])

print(model_accuracy)

#plot model accuracy in one line chart (x-axis: year, y-axis: accuracy) years are 3,4,5,6,7,8,9,10,11
years = [3,4,5,6,7,8,9,10,11]
plt.figure(figsize=(10, 5))
plt.title('Model Accuracy')
plt.xlabel('Year')
plt.ylabel('Accuracy')
for model_name in model_accuracy:
    plt.plot(years, model_accuracy[model_name], label=model_name)
plt.legend()
plt.savefig('output/rolling_window/model_accuracy.png')
plt.show()



    


{'RandomForestClassifier': [0.5229357798165137, 0.41284403669724773, 0.40404040404040403, 0.37777777777777777, 0.4392523364485981, 0.4074074074074074, 0.42201834862385323, 0.4117647058823529, 0.0], 'GradientBoostingClassifier': [0.5229357798165137, 0.4036697247706422, 0.40404040404040403, 0.37777777777777777, 0.40186915887850466, 0.3333333333333333, 0.3853211009174312, 0.3235294117647059, 0.0], 'GaussianNB': [0.5229357798165137, 0.43119266055045874, 0.3939393939393939, 0.37777777777777777, 0.40186915887850466, 0.32407407407407407, 0.3853211009174312, 0.3235294117647059, 0.0], 'LogisticRegression': [0.5229357798165137, 0.5871559633027523, 0.5858585858585859, 0.6333333333333333, 0.616822429906542, 0.6759259259259259, 0.6146788990825688, 0.6764705882352942, 0.0], 'AdaBoostClassifier': [0.5229357798165137, 0.4036697247706422, 0.40404040404040403, 0.37777777777777777, 0.40186915887850466, 0.32407407407407407, 0.3853211009174312, 0.3235294117647059, 0.0]}
